Theano : The Basics
========================

Theano is an optimizing compiler for symbolic math expressions.

( Credit for this workbook : Eben Olson :: https://github.com/ebenolson/pydata2015 )

In [1]:
import theano
import theano.tensor as T

Symbolic variables
==========
Rather than manipulate values directly, Theano has to be able to build up the answer from the inputs - so that it can then optimise how to get to the solution (using the CPU or GPU as efficiently as possible).  Therefore, the basic building blocks are 'symbolic variables' (as opposed to variables with concrete values) :

In [2]:
x = T.scalar()

x

Variables can be used in expressions, but (IMPORTANT!) the result is symbolic as well :

In [5]:
y = 3*(x**2) + 1

type(y)

### Investigating expressions

In [6]:
print(y)

In [7]:
theano.pprint(y)

In [8]:
theano.printing.debugprint(y)

In [9]:
from IPython.display import SVG
SVG(theano.printing.pydotprint(y, return_image=True, format='svg'))

Evaluating expressions
============

Supply a `dict` mapping variables to values

In [10]:
y.eval({x: 2})

Or compile a function

In [11]:
f = theano.function([x], y)

f(2)

Compiled function has been transformed

In [13]:
SVG(theano.printing.pydotprint(f, return_image=True, format='svg'))

Other tensor types
==========

In [14]:
X = T.vector()
X = T.matrix()
X = T.tensor3()
X = T.tensor4()

Numpy style indexing
===========

In [15]:
X = T.vector()

In [16]:
X[1:-1:2]

In [17]:
X[ [1,2,3] ]

Many functions/operations are available through `theano.tensor` or variable methods

In [18]:
y = X.argmax()

In [19]:
y = T.cosh(X)

In [20]:
y = T.outer(X, X)

But don't try to use numpy functions on Theano variables. Results may vary!

Automatic differention
============
- Gradients are free!

In [21]:
x = T.scalar()
y = T.log(x)

In [22]:
gradient = T.grad(y, x)
gradient.eval({x: 2})

# Shared Variables

- Symbolic + Storage

In [23]:
import numpy as np
x = theano.shared(np.zeros((2, 3), dtype=theano.config.floatX))

In [24]:
x

We can get and set the variable's value

In [25]:
values = x.get_value()
print(values.shape)
print(values)

In [26]:
x.set_value(values)

Shared variables can be used in expressions as well

In [27]:
(x + 2) ** 2

Their value is used as input when evaluating

In [28]:
((x + 2) ** 2).eval()

In [29]:
theano.function([], (x + 2) ** 2)()

# Updates

- Store results of function evalution
- `dict` mapping shared variables to new values

In [30]:
count = theano.shared(0)
new_count = count + 1
updates = {count: new_count}

f = theano.function([], count, updates=updates)

In [31]:
f()

In [32]:
f()

In [33]:
f()